<a href="https://colab.research.google.com/github/hamagami/anomaly-detection/blob/main/05_01_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG16ネットワークで抽出した画像特徴をもとに簡単な異常検知を行う

色々なギター画像に対し，ギターに似た画像を異常検知する

In [ ]:
# データ取得　画像をたくさんとってきます
!wget https://dl.dropbox.com/s/dmjzsrqa9s22joi/imgs.zip
!unzip -d . imgs.zip

In [ ]:
from keras.applications import VGG16
from keras.layers import Dense, Dropout, Activation, Flatten
from keras import models, optimizers, layers
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
from keras.applications.resnet50 import preprocess_input
from sklearn.svm import OneClassSVM

import glob  
from PIL import Image 
import numpy as np  
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.image as mpimg
import random

## 画像のサイズを調整

In [ ]:
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

## VGG16ネットワークを取得

In [ ]:
model = VGG16(weights='imagenet', include_top=False, pooling="avg")

## 画像から特徴を取り出す関数

In [ ]:
def feature_extraction(model, img_path, image_size=224):
    im=load_img(f)
    img = expand2square(im, (128, 128, 128)).resize((image_size, image_size))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)  # add a dimention of samples
    x = preprocess_input(x)  # RGB 2 BGR and zero-centering by mean pixel based on the position of channels

    feat = model.predict(x)  # Get image features
    feat = feat.flatten()  # Convert 3-dimentional matrix to (1, n) array

    return feat

## ギター画像の特徴抽出

In [ ]:
data=[]
files = glob.glob("./imgs/guitar/*") 
# サンプル画像
for f in random.sample(files, 3):
  print(f)
  plt.imshow(mpimg.imread(f))
  plt.show()

In [ ]:
# 特徴抽出
for f in files:
        print(f)
        ret= feature_extraction(model, f, 224)
        data.append(ret)

In [ ]:
plt.plot(data[0],label="feature value")
plt.show()

In [ ]:
data = np.array(data)
pca = PCA(n_components=2)
pdata =np.array(pca.fit_transform(data))
plt.scatter(pdata.T[0],pdata.T[1])
plt.show()

In [ ]:
mmscaler = preprocessing.MinMaxScaler() 
mmscaler.fit(data)  
sdata=mmscaler.transform(data)

ocsvm = OneClassSVM(nu=0.001, kernel="rbf", gamma='auto')
ocsvm.fit(sdata)
y_pred = ocsvm.decision_function(sdata).ravel()
plt.plot(y_pred)
plt.show()

In [ ]:
adata=[]
files = glob.glob("./imgs/others/*")   
for f in files:
  print(f)
  plt.imshow(mpimg.imread(f))
  plt.show()  
for f in files:
        print(f)
        ret= feature_extraction(model, f, 224)
        adata.append(ret)
adata = np.array(adata)
sadata=mmscaler.transform(adata)
apred=ocsvm.predict(sadata)
a_pca_data =np.array(pca.fit_transform(adata))

In [ ]:
plt.plot(adata[0])
plt.show()

In [ ]:
plt.scatter(pdata.T[0],pdata.T[1])
plt.scatter(a_pca_data.T[0][0],a_pca_data.T[1][0],label="hyotan",marker="D")
plt.scatter(a_pca_data.T[0][1],a_pca_data.T[1][1],label="neko",marker="D")
plt.scatter(a_pca_data.T[0][4],a_pca_data.T[1][4],label="elec",marker="D")
plt.scatter(a_pca_data.T[0][6],a_pca_data.T[1][6],label="illust",marker="D")
plt.scatter(a_pca_data.T[0][7],a_pca_data.T[1][7],label="piano",marker="D")
plt.legend()
plt.show()